In [2]:
import os

# coloca as variáveis de ambiente necessárias para o PySpark no windows
os.environ['JAVA_HOME'] = r"C:\Program Files\Eclipse Adoptium\jdk-8.0.452.9-hotspot"
os.environ['SPARK_HOME'] = r"C:\spark\spark-3.5.6-bin-hadoop3"

os.environ['PYSPARK_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\diego\Desktop\spark_alura\.venv\Scripts\python.exe"


In [3]:
import findspark # busca o PySpark no sistema
findspark.init()

In [4]:
# importa a classe que inicia uma sessão do Spark para usar o Spark SQL
from pyspark.sql import SparkSession

In [5]:
# inicia uma sessão do Spark, que é o ponto de entrada para usar o Spark SQL
spark = SparkSession.builder.master('local[*]').appName("Iniciando").config("spark.hadoop.io.native.lib", "false").getOrCreate()
spark

In [ ]:
# exemplo de criação de um DataFrame usando tuplas
data = [('zeca', '35'), ('eva', '29')]
col_names = ['nome', 'idade']

df = spark.createDataFrame(data, col_names)

df.show()

+----+-----+
|nome|idade|
+----+-----+
|zeca|   35|
| eva|   29|
+----+-----+



In [17]:
# exemplo de criação de um DataFrame usando dicionários
data_dict = [{'nome': 'zeca', 'idade': '35'}, {'nome': 'eva', 'idade': '29'}]

df = spark.createDataFrame(data_dict)

df.show()

+-----+----+
|idade|nome|
+-----+----+
|   35|zeca|
|   29| eva|
+-----+----+



In [7]:
# converte o DataFrame do Spark para um DataFrame do Pandas
df.toPandas()

,nome,idade
0,zeca,35
1,eva,29


In [ ]:
zip_path = os.path.abspath(r'.\zip_files')
data_path = os.path.abspath(r'.\data')

In [ ]:
import zipfile # biblioteca nativa do Python para manipular arquivos zip

zipfile.ZipFile(os.path.join(zip_path, 'empresas.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'estabelecimentos.zip'), 'r').extractall(data_path)
zipfile.ZipFile(os.path.join(zip_path, 'socios.zip'), 'r').extractall(data_path)

In [ ]:
# retorna uma lista com os caminhos dos arquivos de um diretório
def get_files_from_directory(directory):
    files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
    ]
    return files

In [23]:
# lê todos os arquivos CSV do diretório especificado e cria um DataFrame
df_empresa = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'empresas')), 
    sep=";", header=False, inferSchema=True)

df_empresa.count()

4585679

In [24]:
df_estabelecimentos = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'estabelecimentos')), 
    sep=";", header=False, inferSchema=True)

df_estabelecimentos.count()

4836219

In [25]:
df_socios = spark.read.csv(
    get_files_from_directory(os.path.join(data_path, 'socios')), 
    sep=";", header=False, inferSchema=True)

df_socios.count()

2046430

In [ ]:
# exibe as primeiras 3 linhas dos DataFrames
df_empresa.show(3)
df_estabelecimentos.show(3)
df_socios.show(3)

+----+--------------------+----+---+----+---+----+
| _c0|                 _c1| _c2|_c3| _c4|_c5| _c6|
+----+--------------------+----+---+----+---+----+
| 306|FRANCAMAR REFRIGE...|2240| 49|0,00|  1|NULL|
|1355|BRASILEIRO & OLIV...|2062| 49|0,00|  5|NULL|
|4820|REGISTRO DE IMOVE...|3034| 32|0,00|  5|NULL|
+----+--------------------+----+---+----+---+----+
only showing top 3 rows

+----+---+---+---+--------------+---+--------+---+----+----+--------+-------+----+----+---------------+----+-------+------------------+-------+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|           _c4|_c5|     _c6|_c7| _c8| _c9|    _c10|   _c11|_c12|_c13|           _c14|_c15|   _c16|              _c17|   _c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|
+----+---+---+---+--------------+---+--------+---+----+----+--------+-------+----+----+---------------+----+-------+------------------+-------+----+----+----+----+----+----+----+----+----+----+----+
|1879|  1| 96|  1|PIR

In [ ]:
# converte as primeiras 3 linhas do DataFrame do Spark para um DataFrame do Pandas
df_socios.limit(3).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
